<a href="https://colab.research.google.com/github/jumadi-cloud/Fundamental-Python/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Library yang kita butuhkan

- Pada perintah di bawah kami mengimport semua kebutuhan library yang kami butuhkan,
- Jika temen temen tidak mau import semua library yang di butuhkan dalam step 1 temen temen cukup import library json saja untuk menghubungkan dataset yang sudah kita buat.

### Perlu di note sebelumnya di virtual environment kami sudah Install and import library yang di butuhkan seperti **tensorflow, keras, keras-models, pickle, nltk**

Cara install:

- pip install tensorflow
- pip install keras
- pip install keras-models
- pip install pickle
- pip install nltk



In [3]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("../dataset/intents.json").read()
intents = json.loads(data_file)

# 2. Data pre-processing

In [5]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...


### Tokenisasi

- Pada proses tokenisasi pada dasarnya adalah pemisahan kalimat, paragraf, 
atau seluruh dokumen teks menjadi unit yang lebih kecil, proses itu yang disebut token

- Pada proses ini juga akan save documen tersebut menjadi file label.pkl dan texts.pkl (proses labeling)

In [7]:
nltk.download('omw-1.4')

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('label.pkl','wb'))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...


75 documents
38 classes ['goodbye', 'greeting', 'manfaat_IISMA', 'manfaat_PMMB', 'manfaat_kampus_mengajar', 'manfaat_magang', 'manfaat_pejuang_muda', 'manfaat_pertukaran_mahasiswa', 'manfaat_studi_independen', 'mitra_IISMA', 'mitra_PMMB', 'mitra_magang', 'mitra_studi_independen', 'penjelasan_IISMA', 'penjelasan_PMMB', 'penjelasan_kampus_mengajar', 'penjelasan_kampus_merdeka', 'penjelasan_magang', 'penjelasan_pejuang_muda', 'penjelasan_pertukaran_mahasiswa', 'penjelasan_studi_independen', 'periode_dan_pendaftaran_IISMA', 'periode_dan_pendaftaran_PMMB', 'periode_dan_pendaftaran_kampus_mengajar', 'periode_dan_pendaftaran_magang', 'periode_dan_pendaftaran_pejuang_muda', 'periode_dan_pendaftaran_pertukaran_mahasiswa', 'periode_dan_pendaftaran_studi_independen', 'persyaratan_IISMA', 'persyaratan_kampus_mengajar', 'persyaratan_kampus_merdeka', 'persyaratan_magang', 'persyaratan_pejuang_muda', 'persyaratan_pertukaran_mahasiswa', 'persyaratan_studi_independen', 'program_kampus_merdeka', 'terima

# 3. Creating Training Data

- Pada dasarnya, bag of words adalah representasi sederhana dari setiap teks dalam sebuah kalimat sebagai bag of words-nya.

In [8]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


# 5. Creating Modeling 

- Pada proses ini kami akan membuat model jaringan saraf dan menyimpan model tersebut 

In [9]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
15/15 [==============================] - 1s 1ms/step - loss: 3.6799 - accuracy: 0.0267 
Epoch 2/200
15/15 [==============================] - 0s 1ms/step - loss: 3.5365 - accuracy: 0.1600
Epoch 3/200
15/15 [==============================] - 0s 1ms/step - loss: 3.3644 - accuracy: 0.1733
Epoch 4/200
15/15 [==============================] - 0s 1ms/step - loss: 3.2108 - accuracy: 0.2000
Epoch 5/200
15/15 [==============================] - 0s 1ms/step - loss: 2.9793 - accuracy: 0.2400
Epoch 6/200
15/15 [==============================] - 0s 1ms/step - loss: 2.9115 - accuracy: 0.2667
Epoch 7/200
15/15 [==============================] - 0s 1ms/step - loss: 2.7011 - accuracy: 0.2533
Epoch 8/200
15/15 [==============================] - 0s 1ms/step - loss: 2.6562 - accuracy: 0.2000
Epoch 9/200
15/15 [==============================] - 0s 1ms/step - loss: 2.6110 - accuracy: 0.1467
Epoch 10/200
15/15 [==============================] - 0s 1ms/step - loss: 2.4094 - accuracy: 0.3200
Epoch 11

15/15 [==============================] - 0s 1ms/step - loss: 0.4379 - accuracy: 0.8667
Epoch 83/200
15/15 [==============================] - 0s 1ms/step - loss: 0.4398 - accuracy: 0.8267
Epoch 84/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5095 - accuracy: 0.8667
Epoch 85/200
15/15 [==============================] - 0s 1ms/step - loss: 0.3534 - accuracy: 0.8667
Epoch 86/200
15/15 [==============================] - 0s 994us/step - loss: 0.4648 - accuracy: 0.8667
Epoch 87/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5243 - accuracy: 0.8267
Epoch 88/200
15/15 [==============================] - 0s 1ms/step - loss: 0.3668 - accuracy: 0.8400
Epoch 89/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5194 - accuracy: 0.8267
Epoch 90/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5871 - accuracy: 0.8133
Epoch 91/200
15/15 [==============================] - 0s 1ms/step - loss: 0.4552 - accuracy: 0.8400
Epoch 92/20

15/15 [==============================] - 0s 941us/step - loss: 0.1681 - accuracy: 0.9467
Epoch 163/200
15/15 [==============================] - 0s 925us/step - loss: 0.3836 - accuracy: 0.9067
Epoch 164/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0931 - accuracy: 0.9733
Epoch 165/200
15/15 [==============================] - 0s 854us/step - loss: 0.3772 - accuracy: 0.8667
Epoch 166/200
15/15 [==============================] - 0s 854us/step - loss: 0.1858 - accuracy: 0.9600
Epoch 167/200
15/15 [==============================] - 0s 926us/step - loss: 0.1781 - accuracy: 0.9467
Epoch 168/200
15/15 [==============================] - 0s 964us/step - loss: 0.3617 - accuracy: 0.8933
Epoch 169/200
15/15 [==============================] - 0s 925us/step - loss: 0.2147 - accuracy: 0.9200
Epoch 170/200
15/15 [==============================] - 0s 925us/step - loss: 0.1453 - accuracy: 0.9600
Epoch 171/200
15/15 [==============================] - 0s 1ms/step - loss: 0.2776 - accur